# Predicting Crypto prices
1. Get historic stock prices
2. Extract prices to predict
3. Normalise
4. Create Time Series
5. Build model
6. Fit model
7. Test and evaluate

# X and Y
X = Goes back looking over a series of time e.g. past 7 minutes.<br>
Y = The last nth of the series of time e.g. the price of BTC on the 7th minute of the past 7 minutes.<br>

In [ ]:
# Example
"""
look_back = 3
data = [1,2,4,5,7,8,9]

   X      Y
[1,2,4]->[5]
[2,4,5]->[7]
[4,5,7]->[8]
[5,7,8]->[9]
"""

# Import dependencies

In [ ]:
# install dependencies
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

# Load dataset

In [ ]:
# 1. Get historic stock prices
def allData():
    path = '../datasets/BTC/'
    
    ori_df = pd.read_csv(f"{path}gemini_BTCUSD_2015_1min.csv")
    for item in range(2016, 2022):
        df = pd.read_csv(f"{path}/gemini_BTCUSD_{item}_1min.csv")
        new_df = pd.concat([ori_df, df], ignore_index=True)
        ori_df = new_df

    ori_df.rename({"Unix Timestamp": "Timestamp"}, axis=1, inplace=True)
    combine_df = ori_df
    combine_df.drop_duplicates(subset=["Timestamp"], keep='first', inplace=True)
    
    return combine_df

data = allData()

# Explore dataset

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns

In [ ]:
data = data.drop(columns=['Timestamp', 'Symbol', 'Open', 'High', 'Low', 'Volume'])

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_axis(data['Date'], inplace=True)
data.set_index('Date', inplace=True)

In [ ]:
data.sort_index(inplace=True)

In [ ]:
data.info()

In [ ]:
data

# Data visualisation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data['Close']

In [ ]:
data_close = data['Close'].values
data_close

In [ ]:
data_close = data_close.reshape((-1, 1))
plt.plot(data_close)
plt.show()

# Split data into train and test

In [ ]:
train_size = int(len(data_close) * 0.8) # 80% of data will be for training

data_train = data_close[:train_size]
data_test = data_close[train_size:] # the rest will be for testing

print('Train data size: ', len(data_train))
print('Test data size: ', len(data_test))

# Create time series out of data

In [ ]:
look_back = 15 # look back to the past 60 minutes (one hour) of the prices

train_generator = TimeseriesGenerator(data_train, data_train, length=look_back, batch_size=64)     
test_generator = TimeseriesGenerator(data_test, data_test, length=look_back, batch_size=1)

# Set up and build LSTM model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import L2

In [ ]:
# initialise model
model = Sequential()
# add layers
model.add(LSTM(50, activation='relu', input_shape=(look_back, 1), recurrent_dropout=0.2))
#model.add(Dropout(0.2, input_shape=(look_back, 1)))
model.add(Dense(1))
# compile model
model.compile(loss='mse', optimizer='adam')

In [ ]:
# fit the model
num_epochs = 20
model.fit(train_generator, epochs=num_epochs, verbose=1)

In [ ]:
prediction = model.predict(test_generator)

data_train = data_train.reshape((-1))
data_test = data_test.reshape((-1))
prediction = prediction.reshape((-1))

In [ ]:
# Plot the predictions on a graph
prediction_plot = np.empty_like(data_close.reshape((-1)))
prediction_plot[:] = np.nan
prediction_plot[(len(data_train) + look_back):] = prediction

plt.plot(data_close)
plt.plot(prediction_plot)
plt.show()

# Make forecast

In [ ]:
data_close = data_close.reshape((-1))

def predict(num_prediction, model):
    prediction_list = data_close[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list

num_prediction = 100
forecast = predict(num_prediction, model)

In [ ]:
forecast = forecast.reshape((-1))
plt.plot(forecast)
plt.show()

In [ ]:
data_forecast = np.append(data_close, forecast)

In [ ]:
# Plot the predictions on a graph
forecast_plot = np.empty_like(data_forecast)
forecast_plot[:] = np.nan
forecast_plot[len(data_close):] = forecast

plt.plot(data_close)
plt.plot(forecast_plot)
plt.show()